# Module 8: 8 - Anatomy of an AI Agent - Validating Tool Arguments with Pydantic
---------------------------------------------------------------------------------
In this lesson, we will enhance our AI Agent by validating the suggested tool arguments provided by the LLM using Pydantic, a popular data validation library for Python. This ensures that the arguments match the Python function signatures before the tool execution process. We will leverage OpenAI's Function Calling capability for the AI models to detect when a tool needs to be used based on user intent, choose the right tool, and return the function signature to be executed by the AI agent.

## Objectives
* Understand the concept of validating tool arguments using Pydantic.
* Update the LLM Client and agent to support tool execution with argument validation.
* Define and use Pydantic models to validate function arguments.
* Test the AI Agent's ability to validate and execute tools based on user input.

## What this session covers:
* Defining the current agent structure, including the LLM Client and short-term memory.
* Implementing OpenAI's Function Calling for tool execution.
* Defining dummy tools and their function signatures.
* Updating the agent to include tools and validate their arguments using Pydantic.
*Initializing and testing the enhanced AI Agent with validated tool execution.

## Install Libraries

In [2]:
#! pip install openai

## Define Current Agent Structure

### LLM Client

In [3]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

### Short-Term Memory

In [4]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""
    
    def __init__(self):
        self.messages = []
    
    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)
    
    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)
    
    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()
    
    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

### Agent Base

In [5]:
from typing import Dict

class Agent:
    """Integrates LLM client, tools, and memory."""
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history, tools=self.tools)
        self.memory.add_message(response)
        return response

## Defining Dummy Tools

In [6]:
import random

def get_weather(location: str) -> str:
    """Gets weather information."""
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

In [7]:
get_weather_func_dict = {
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get weather information based on location.',
        'parameters': {
            'properties': {'location': {'type': 'string'}},
            'required': ['location'],
            'type': 'object'
        }
    }
}

jump_func_dict = {
    'type': 'function',
    'function': {
        'name': 'jump',
        'description': 'Jump a specific distance.',
        'parameters': {
            'properties': {'distance': {'type': 'string'}},
            'required': ['distance'],
            'type': 'object'
        }
    }
}

tools = [get_weather_func_dict, jump_func_dict]

## Initialize Client

In [8]:
# API from environment variable
# import os
# api_key = os.getenv("OPENAI_API_KEY"))

api_key=""

client = OpenAIChatCompletion(
    base_url='https://api.openai.com/v1',
    model='gpt-4',
    api_key=api_key
)

## Define System messages

In [9]:
# Define the system message
system_message = {"role": "system", "content": "You are a weather assistant."}

## Initialize Agent

In [10]:
# Initialize the Agent with the LLM client, system message, and tools
agent = Agent(llm_client=client, system_message=system_message, tools=tools)

## Send a User Message

In [11]:
# Define a user message
user_message = {"role": "user", "content": "What is the weather in Virginia?"}

# Generate a response using the agent
response = agent.run(user_message)
response

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SKljCd3EefmcpudNRN7SyouF', function=Function(arguments='{\n  "location": "Virginia"\n}', name='get_weather'), type='function')])

In [12]:
response.to_dict()

{'content': None,
 'role': 'assistant',
 'tool_calls': [{'id': 'call_SKljCd3EefmcpudNRN7SyouF',
   'function': {'arguments': '{\n  "location": "Virginia"\n}',
    'name': 'get_weather'},
   'type': 'function'}]}

## Parse Tool Response

In [13]:
import json

tool_response = response.tool_calls[0]
tool_arguments = json.loads(tool_response.function.arguments)
tool_arguments

{'location': 'Virginia'}

In [19]:
tool_response = response.tool_calls[0]
tool_arguments_json = tool_response.function.arguments
tool_arguments_json

'{\n  "location": "Virginia"\n}'

## Validate Function Signature

### Define Pydantic Model

In [14]:
import random

def get_weather(location: str) -> str:
    """Gets weather information."""
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

# Pydantic Model
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    """Get weather information based on location."""
    location: str = Field(description="Location to get weather for")

### Validate Tool Arguments - JSON Format

In [20]:
response_model = GetWeatherSchema.model_validate_json(tool_arguments_json)
response_model

GetWeatherSchema(location='Virginia')

### Convert to Dictionary

In [21]:
tool_arguments_dict = response_model.model_dump()
tool_arguments_dict

{'location': 'Virginia'}

## Execute Tool

In [22]:
get_weather(**tool_arguments_dict)

'Virginia: 63F.'